In [11]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
import pandas as pd
import numpy as np
import json
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split, GridSearchCV

In [4]:
# file = open('Antidepressants.txt', encoding='utf-8') 
with open('C:/Users/User/Desktop/al/учёба/курсовая/all_reviews_texts_318352.txt', encoding='UTF8') as file:
    reviews = pd.DataFrame()
    for i in range(12500):
        line = next(file)
        pik = json.loads(line)
        reviews = reviews.append(pd.DataFrame.from_dict(pik))  

In [5]:
reviews.head(5)

,url,cat1,cat2,cat3,cat4,rating_1,rating_2,rating_3,rating_4,rating_5,...,title,author,author_location,authorid,recom_author_mark,overall,description,product_name_url,product_name_title,product-rating
0,https://otzovik.com/review_1650685.html,Красота и здоровье,Лекарственные средства,Разное,Solvay Pharma,width: 100%;,width: 100%;,width: 100%;,width: 100%;,width: 100%;,...,Отзыв: Таблетки от головокружения Бетасерк - П...,Любовная любовь,"Россия, Рязань",/profile/%D0%9B%D1%8E%D0%B1%D0%BE%D0%B2%D0%BD%...,ДА,"Помогает при головокружении, только нужно прин...",У меня последний год стала кружиться голова. Я...,https://otzovik.com/reviews/preparat_solvay_ph...,Все отзывы о Таблетки от головокружения Бетасерк,Общий рейтинг: 5
0,https://otzovik.com/review_5260629.html,В городе,Больницы и клиники,Детские больницы,,width: 100%;,width: 100%;,width: 100%;,width: 100%;,width: 100%;,...,Отзыв: Луганская областная детская клиническая...,Аноним1383147,"Украина, Луганск",/profile/%D0%90%D0%BD%D0%BE%D0%BD%D0%B8%D0%BC1...,ДА,замечательная больница.отдельное спасибо онког...,Я мать ребёнка который попал в эту больницу. и...,https://otzovik.com/reviews/luganskaya_oblastn...,Все отзывы о Луганская областная детская клини...,Общий рейтинг: 5
0,https://otzovik.com/review_6665469.html,Красота и здоровье,Лекарственные средства,Ноотропные препараты,EGIS,width: 100%;,width: 100%;,width: 100%;,width: 100%;,width: 100%;,...,"Отзыв: Ноотропное средство EGIS ""Луцетам"" - Эф...",maxzykov90,"Россия, Барнаул",/profile/maxzykov90,ДА,"Эффективно, однако нужны высокие дозы",Обычный пирацетам выпускается в максимальной д...,https://otzovik.com/reviews/nootropnoe_sredstv...,"Все отзывы о Ноотропное средство EGIS ""Луцетам""",Общий рейтинг: 5
0,https://otzovik.com/review_544236.html,Красота и здоровье,Альтернативная медицина,Гомеопатические препараты,Bionorica,width: 100%;,width: 100%;,width: 100%;,width: 100%;,,...,Отзыв: Гомеопатический препарат Циклодинон - т...,natalipon,"Россия, Нижний Новгород",/profile/natalipon,ДА,теперь мой организм работает регулярно,Циклодинон мне прописал врач для нормализации ...,https://otzovik.com/reviews/gomeopaticheskiy_p...,Все отзывы о Гомеопатический препарат Циклодинон,Общий рейтинг: 5
0,https://otzovik.com/review_3322356.html,Красота и здоровье,Альтернативная медицина,БАДы,Фелицата Холдинг,width: 20%;,width: 60%;,width: 60%;,width: 80%;,,...,Отзыв: БАД Фелицата Холдинг Прелакс - Совершен...,karines,"Украина, Ивано-Франковск",/profile/karines,,Совершенно без полезное,Всем доброго времени суток. Сегодня хочу с ва...,https://otzovik.com/reviews/bad_felicata_holdi...,Все отзывы о БАД Фелицата Холдинг Прелакс,Общий рейтинг: 2


In [6]:
reviews = reviews.replace(['width: 20%;', 'width: 40%;', 'width: 60%;', 'width: 80%;', 'width: 100%;'], [1, 2, 3, 4, 5])
reviews = reviews.replace(['Общий рейтинг: 1', 'Общий рейтинг: 2', 'Общий рейтинг: 3', 'Общий рейтинг: 4', 'Общий рейтинг: 5'], [1, 2, 3, 4, 5])

In [7]:
reviews_part = pd.DataFrame()
for i in range(1,6):
    reviews_part = reviews_part.append(reviews[reviews['product-rating']==i].head(500))
reviews_part.head()

,url,cat1,cat2,cat3,cat4,rating_1,rating_2,rating_3,rating_4,rating_5,...,title,author,author_location,authorid,recom_author_mark,overall,description,product_name_url,product_name_title,product-rating
0,https://otzovik.com/review_3514847.html,В городе,Больницы и клиники,Акушерство и гинекология,"Россия, Московская область",1,1,1,1,1,...,"Отзыв: Роддом г. Люберцы (Россия, Люберцы) - У...",Аноним853156,"Россия, Москва",/profile/%D0%90%D0%BD%D0%BE%D0%BD%D0%B8%D0%BC8...,,Ужас! Не рожайте там,Хочу оставить свой отзыв дабы огородить от это...,https://otzovik.com/reviews/roddom_g_lyuberci_...,"Все отзывы о Роддом г. Люберцы (Россия, Люберцы)",1
0,https://otzovik.com/review_5640852.html,В городе,Больницы и клиники,Диагностические центры и лаборатории,,1,1,1,1,1,...,"Отзыв: Лаборатория ""Гемотест"" - ужасно и некко...",Кристина Вайт,"Россия, Москва",/profile/%D0%9A%D1%80%D0%B8%D1%81%D1%82%D0%B8%...,,ужасно и неккоректно,На протяжении всей беременности сдавала анализ...,https://otzovik.com/reviews/laboratoriya_gemot...,"Все отзывы о Лаборатория ""Гемотест""",1
0,https://otzovik.com/review_3860601.html,В городе,Больницы и клиники,Поликлиники,"Россия, Москва",1,1,1,1,1,...,Отзыв: Городская клиническая больница им. С.П....,Avtostop,"Россия, Москва",/profile/Avtostop,,Приёмный покой - нет слов! Одни эмоции!,Безалаберность в приёмном покое! 4.10.16 около...,https://otzovik.com/reviews/gorodskaya_klinich...,Все отзывы о Городская клиническая больница им...,1
0,https://otzovik.com/review_3215475.html,Красота и здоровье,Альтернативная медицина,Гомеопатические препараты,Сашера-Мед,1,3,1,1,,...,Отзыв: Капли растительные Сашера-Мед Intoxic -...,Добрая весна,Россия,/profile/%D0%94%D0%BE%D0%B1%D1%80%D0%B0%D1%8F+...,,Всего понемножку. И с чего глистам пугаться?) ...,Капли Интоксик - новомодное отечественное сред...,https://otzovik.com/reviews/kapli_rastitelnie_...,Все отзывы о Капли растительные Сашера-Мед Int...,1
0,https://otzovik.com/review_4210625.html,В городе,Больницы и клиники,Диагностические центры и лаборатории,,3,5,4,1,1,...,"Отзыв: Медицинский диагностический центр ""Рэмс...",кукузя1000,"Россия, Санкт-Петербург",/profile/%D0%BA%D1%83%D0%BA%D1%83%D0%B7%D1%8F1000,,"Разводят на ненужные услуги, дорого","Очень разочарована, как и почему медицинская к...",https://otzovik.com/reviews/medicinskiy_diagno...,Все отзывы о Медицинский диагностический центр...,1


In [8]:
reviews_part['Positively_Rated'] = np.where(reviews_part['product-rating']>=3, 1, 0)
reviews_part.head()

,url,cat1,cat2,cat3,cat4,rating_1,rating_2,rating_3,rating_4,rating_5,...,author,author_location,authorid,recom_author_mark,overall,description,product_name_url,product_name_title,product-rating,Positively_Rated
0,https://otzovik.com/review_3514847.html,В городе,Больницы и клиники,Акушерство и гинекология,"Россия, Московская область",1,1,1,1,1,...,Аноним853156,"Россия, Москва",/profile/%D0%90%D0%BD%D0%BE%D0%BD%D0%B8%D0%BC8...,,Ужас! Не рожайте там,Хочу оставить свой отзыв дабы огородить от это...,https://otzovik.com/reviews/roddom_g_lyuberci_...,"Все отзывы о Роддом г. Люберцы (Россия, Люберцы)",1,0
0,https://otzovik.com/review_5640852.html,В городе,Больницы и клиники,Диагностические центры и лаборатории,,1,1,1,1,1,...,Кристина Вайт,"Россия, Москва",/profile/%D0%9A%D1%80%D0%B8%D1%81%D1%82%D0%B8%...,,ужасно и неккоректно,На протяжении всей беременности сдавала анализ...,https://otzovik.com/reviews/laboratoriya_gemot...,"Все отзывы о Лаборатория ""Гемотест""",1,0
0,https://otzovik.com/review_3860601.html,В городе,Больницы и клиники,Поликлиники,"Россия, Москва",1,1,1,1,1,...,Avtostop,"Россия, Москва",/profile/Avtostop,,Приёмный покой - нет слов! Одни эмоции!,Безалаберность в приёмном покое! 4.10.16 около...,https://otzovik.com/reviews/gorodskaya_klinich...,Все отзывы о Городская клиническая больница им...,1,0
0,https://otzovik.com/review_3215475.html,Красота и здоровье,Альтернативная медицина,Гомеопатические препараты,Сашера-Мед,1,3,1,1,,...,Добрая весна,Россия,/profile/%D0%94%D0%BE%D0%B1%D1%80%D0%B0%D1%8F+...,,Всего понемножку. И с чего глистам пугаться?) ...,Капли Интоксик - новомодное отечественное сред...,https://otzovik.com/reviews/kapli_rastitelnie_...,Все отзывы о Капли растительные Сашера-Мед Int...,1,0
0,https://otzovik.com/review_4210625.html,В городе,Больницы и клиники,Диагностические центры и лаборатории,,3,5,4,1,1,...,кукузя1000,"Россия, Санкт-Петербург",/profile/%D0%BA%D1%83%D0%BA%D1%83%D0%B7%D1%8F1000,,"Разводят на ненужные услуги, дорого","Очень разочарована, как и почему медицинская к...",https://otzovik.com/reviews/medicinskiy_diagno...,Все отзывы о Медицинский диагностический центр...,1,0


In [25]:
# X_train, X_test, y_train, y_test = train_test_split(antigroup.drop(['url', 'cat1', 'cat2', 'cat3', 'cat4', 'rating_1', 'rating_2', 'rating_3', 'rating_4', 'rating_5','rating_1_title', 'rating_2_title', 'rating_3_title', 'rating_4_title', 'rating_5_title', 'review_date', 'recom_number', 'author', 'author_location', 'authorid', 'recom_author_mark', 'product_name_url', 'product_name_title', 'product-rating', 'Positively_Rated', 'title', 'plus', 'minus'], axis=1), antigroup['Positively_Rated'], test_size=0.2, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(reviews_part['description'], reviews_part['Positively_Rated'], test_size=0.2, random_state=0)

In [26]:
vectorizer = TfidfVectorizer(min_df = 5, max_features = 50000)
tf_transformer = TfidfTransformer(use_idf=True)
X_counts = vectorizer.fit_transform(X_train)
X_tfidf = tf_transformer.fit_transform(X_counts)

In [27]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
clf = LogisticRegression().fit(X_tfidf, y_train)
predicted = clf.predict(X_tfidf)

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [29]:
print(metrics.classification_report(y_train, predicted))
print('MSE: ', metrics.mean_squared_error(y_train, predicted))

              precision    recall  f1-score   support

           0       0.97      0.76      0.86       806
           1       0.86      0.99      0.92      1194

    accuracy                           0.90      2000
   macro avg       0.92      0.87      0.89      2000
weighted avg       0.91      0.90      0.89      2000

MSE:  0.104


In [30]:
X_new_counts = vectorizer.transform(X_test)
X_new_tfidf = tf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)

In [31]:
print(metrics.classification_report(y_test, predicted))
print('MSE: ', metrics.mean_squared_error(y_test, predicted))

              precision    recall  f1-score   support

           0       0.73      0.42      0.53       194
           1       0.71      0.90      0.79       306

    accuracy                           0.71       500
   macro avg       0.72      0.66      0.66       500
weighted avg       0.72      0.71      0.69       500

MSE:  0.286


In [32]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(min_df = 5, max_features = 50000)),
    ('clf', LogisticRegression()),
])
parameters = {
    'tfidf__use_idf': (True, False),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]
}
grid_search_tune = GridSearchCV(pipeline, parameters)
grid_search_tune.fit(X_train, y_train)

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\user\app

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\user\appdata\local\programs\python\python37\lib\site-pa

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\user\appdata\local\programs\python\python37\lib\site-pa

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=50000,
                                                        min_df=5,
                                                        ngram_range=(1, 1),
                                   

In [33]:
print("Best parameters set:")
print(grid_search_tune.best_params_)
print('\n', grid_search_tune.best_estimator_.steps)

Best parameters set:
{'clf__C': 10.0, 'tfidf__ngram_range': (1, 3), 'tfidf__use_idf': True}

 [('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=50000,
                min_df=5, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)), ('clf', LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False))]


In [34]:
best_clf = grid_search_tune.best_estimator_
best_clf.fit(X_train, y_train)
predicted = best_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted))
print('MSE: ', metrics.mean_squared_error(y_test, predicted))

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.76      0.59      0.66       194
           1       0.77      0.88      0.82       306

    accuracy                           0.77       500
   macro avg       0.77      0.73      0.74       500
weighted avg       0.77      0.77      0.76       500

MSE:  0.232
